In [5]:
import numpy as np
import pandas as pd
import re

import matplotlib.pyplot as plt
# import seaborn as sns


# import plotly.graph_objects as go
# from plotly.subplots import make_subplots

# import plotly.express as px

In [7]:
from time import perf_counter 


import time
import concurrent.futures

import sys


from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
import pickle


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
invoice = pd.read_csv('/content/drive/My Drive/GreyAtomCapstone/invoice_refined.csv',low_memory=False)

In [10]:
cno_freq = pd.read_csv('/content/drive/My Drive/GreyAtomCapstone/cno_freq.csv',low_memory=False)


In [11]:
invoice_smm_average = pd.read_csv('/content/drive/My Drive/GreyAtomCapstone/invoice_smm_average.csv',low_memory=False)


In [12]:
invoice_smm_average = pd.read_csv('/content/drive/My Drive/GreyAtomCapstone/invoice_smm_average.csv',low_memory=False)

inv_kms_lifetime = pd.read_csv('/content/drive/My Drive/GreyAtomCapstone/inv_kms_lifetime.csv',low_memory=False)

inv_snmm_avg = pd.read_csv('/content/drive/My Drive/GreyAtomCapstone/inv_snmm_avg.csv',low_memory=False)

In [13]:
for df in [invoice,cno_freq,invoice_smm_average,inv_kms_lifetime]:   
    print(df.info())
df=None

inv_kms_lifetime[inv_kms_lifetime['KMs_3L_cap'].between(250000,300000, inclusive=True)].shape

invoice['Frequency_year'] = cno_freq['Frequency_year']

invoice['SMM_average'] = invoice_smm_average['SMM_average']

invoice['kms_lifetime_3L'] = inv_kms_lifetime['kms_lifetime_3L']

# inv_snmm_avg
invoice['SnMM_average'] = inv_snmm_avg['SnMM_average']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 492314 entries, 0 to 492313
Data columns (total 41 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Unnamed: 0           492314 non-null  int64  
 1   Area / Locality      468424 non-null  object 
 2   CITY                 492313 non-null  object 
 3   Cash /Cashless Type  27420 non-null   object 
 4   Claim No.            4804 non-null    object 
 5   Cust Type            492314 non-null  object 
 6   Customer No.         492314 non-null  object 
 7   District             492314 non-null  object 
 8   Expiry Date          22231 non-null   object 
 9   Gate Pass Date       40915 non-null   object 
 10  Gate Pass Time       492314 non-null  object 
 11  Invoice Date         492314 non-null  object 
 12  Invoice No           492314 non-null  int64  
 13  Invoice Time         492314 non-null  object 
 14  Job Card No          492314 non-null  int64  
 15  JobCard Date     

In [14]:



state_map={'Maharashtra': 'West',
 'Tamil Nadu': 'South',
 'Karnataka': 'South',
 'Uttar Pradesh': 'North',
 'Gujarat': 'West',
 'Telangana': 'South',
 'Punjab': 'North',
 'Andhra Pradesh': 'South',
 'Haryana': 'North',
 'Madhya Pradesh': 'West',
 'Rajasthan': 'West',
 'Kerala': 'South',
 'Himachal Pradesh': 'North',
 'Bihar': 'East',
 'Uttarakhand': 'North',
 'Odisha': 'East',
 'Puducherry': 'South',
 'West Bengal': 'East',
 'Chhattisgarh': 'East',
 'Jharkhand': 'East',
 'Assam': 'East',
 'Chandigarh': 'North'}
invoice['state_news'] = invoice['plant_state'].map(state_map)

invoice['state_news'].value_counts(dropna=False)

South    214484
West     183147
North     86793
East       7890
Name: state_news, dtype: int64

In [15]:
# dump df other than invoice

cno_freq = None
invoice_smm_average = None
inv_kms_lifetime = None
inv_snmm_avg = None



In [16]:
# invoice[invoice.Model.isnull()].shape



In [17]:
# invoice.shape


In [18]:

invoice = invoice[invoice.Model.notnull()]

invoice.shape[0]

490789

In [19]:
# temp = invoice[(invoice['SMM_average']<100.0)]
# print(temp.shape)
# print(temp[(temp['SMM_average']==temp['SMM_average']\
#                                        .min())][['Customer No.', 'SMM_average']].head())
# # fig = px.histogram(x=invoice['SMM_average'], barmode='overlay',marginal='box', nbins=100)
# # fig.show()

# temp = invoice[(invoice['SnMM_average']<100.0)]
# print(temp.shape)
# print(temp[(temp['SnMM_average']==temp['SMM_average']\
#                                        .min())][['Customer No.', 'SMM_average']].head())
# # fig = px.histogram(x=invoice['SnMM_average'], barmode='overlay',marginal='box', nbins=100)
# # fig.show()



In [20]:
invoice['SMM_average_no0'] = invoice['SMM_average']
invoice.loc[invoice['SMM_average']<100.0,'SMM_average_no0'] = 100.0

In [21]:


# print_cols('145058', ['Make', 'Model','plant_city','plant_state', 'Total Value','SMM_average'])

invoice[(invoice['SMM_average']<=1000.0)].shape

invoice['SnMM_average_no0'] = invoice['SnMM_average']

invoice.loc[invoice['SnMM_average']<100.0,'SnMM_average_no0'] = 100.0

print(invoice[invoice['SnMM_average_no0']>=100.0].shape[0]+invoice.SnMM_average_no0.isnull().sum() == invoice.shape[0])

#checks out

temp = invoice[(invoice['kms_lifetime_3L']<.1)]
print(temp.shape)

# .1*300000 #KMs life left

invoice['kms_lifetime_3L_no0'] = invoice['kms_lifetime_3L']

invoice.loc[invoice['kms_lifetime_3L']<0.1,'kms_lifetime_3L_no0'] = 0.1

invoice[invoice['kms_lifetime_3L_no0']>=0.1].shape[0] == invoice.shape[0]



True
(1618, 48)


True

In [22]:
temp = invoice.Model.value_counts()
# k=0
# for i in temp.index:
#     print(f'{k}-{i}:{temp[i]}')
#     k +=1
model_list160 = temp[:160].index

invoice['Model_other'] = invoice['Model'].map(lambda x: x if x in model_list160 else 'Other')

invoice['Model_other'].value_counts()

snmm_columns = ['plant_state','state_news','Make','Model_other',
                'KMs Reading','Total Value',
                'Frequency_year','SnMM_average_no0','kms_lifetime_3L_no0']
                
smm_columns = ['plant_state','state_news','Make','Model_other',
                'KMs Reading','Total Value',
                'Frequency_year','SMM_average_no0','kms_lifetime_3L_no0']
invoice_snmm = invoice[snmm_columns]
invoice_smm = invoice[smm_columns]
invoice_smm.info()

# invoice_snmm = invoice_snmm[invoice_snmm.Model.notnull()]

invoice_smm.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 490789 entries, 0 to 492313
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   plant_state          490789 non-null  object 
 1   state_news           490789 non-null  object 
 2   Make                 490789 non-null  object 
 3   Model_other          490789 non-null  object 
 4   KMs Reading          490789 non-null  int64  
 5   Total Value          490789 non-null  float64
 6   Frequency_year       490789 non-null  float64
 7   SMM_average_no0      490789 non-null  float64
 8   kms_lifetime_3L_no0  490789 non-null  float64
dtypes: float64(4), int64(1), object(4)
memory usage: 37.4+ MB


(490789, 9)

In [23]:
# invoice_snmm.isnull().sum()



In [24]:
# LTV calculation

invoice_snmm['LTV_snmm'] = invoice_snmm['Frequency_year']*invoice_snmm['SnMM_average_no0']\
    *invoice_snmm['kms_lifetime_3L_no0']*100

invoice_snmm['LTV_snmm'] = round(invoice_snmm['LTV_snmm'],4)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [25]:

invoice_smm['LTV_smm'] = invoice_smm['Frequency_year']*invoice_smm['SMM_average_no0']\
    *invoice_smm['kms_lifetime_3L_no0']*100

invoice_smm['LTV_smm'] = round(invoice_smm['LTV_smm'],4)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [26]:

# print(invoice_snmm.LTV_snmm.min())
# print(invoice_snmm.LTV_snmm.max())
# print(invoice_snmm.LTV_snmm.mean())
# print(invoice_snmm.LTV_snmm.median())




In [27]:
X = invoice_snmm.drop(['LTV_snmm','Frequency_year',
                       'SnMM_average_no0','kms_lifetime_3L_no0'],axis=1)
# X = invoice_smm.drop(['LTV_smm','Frequency_year',
#                        'SMM_average_no0','kms_lifetime_3L_no0'],axis=1)
print(X.shape)

X.info()

y = invoice_snmm['LTV_snmm']
# y=invoice_smm['LTV_smm']
y.shape

(490789, 6)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 490789 entries, 0 to 492313
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   plant_state  490789 non-null  object 
 1   state_news   490789 non-null  object 
 2   Make         490789 non-null  object 
 3   Model_other  490789 non-null  object 
 4   KMs Reading  490789 non-null  int64  
 5   Total Value  490789 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 26.2+ MB


(490789,)

In [28]:
# one-hot encoding
X = pd.concat([X, pd.get_dummies(X.drop(columns=['KMs Reading','Total Value'],axis=1)
                                 , columns=['plant_state','state_news','Make','Model_other'])], axis=1)

In [29]:
X.drop(columns=['plant_state','state_news','Make','Model_other'], axis=1, inplace=True)

X.shape

(490789, 217)

In [30]:
# invoice.Model_other.nunique()+invoice.Make.nunique()+4

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10)

In [32]:
#Regression

In [33]:
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
xg_reg = xgb.XGBRegressor(objective='reg:squarederror',
                        learning_rate = 0.1,
                        max_depth = 3)

In [ ]:
xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [ ]:
ypred_xgb = xg_reg.predict(X_test)

In [ ]:
# df = X_test.copy()
# df['ypred_xgb'] = ypred_xgb
# df.to_csv('ypred_xgb.csv')
# !cp ypred_xgb.csv "drive/My Drive/GreyAtomCapstone/"
# df=None

In [ ]:
# pickle_out = open("xgb_model.pickle","wb")
# pickle.dump(xg_reg, pickle_out)
# pickle_out.close()
# !cp xgb_model.pickle "drive/My Drive/GreyAtomCapstone/"

---

In [ ]:
rf = RandomForestRegressor()

In [ ]:
rf.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
ypred_rf = rf.predict(X_test)

In [ ]:
# df = X_test.copy()
# df['ypred_rf'] = ypred_rf
# df.to_csv('ypred_rf.csv')
# !cp ypred_rf.csv "drive/My Drive/GreyAtomCapstone/"
# df=None

In [ ]:
# pickle_out = open("rf_model.pickle","wb")
# pickle.dump(rf, pickle_out)
# pickle_out.close()
# !cp rf_model.pickle "drive/My Drive/GreyAtomCapstone/"

---

In [ ]:
etr = ExtraTreesRegressor()

In [ ]:
etr.fit(X_train,y_train)

ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=100, n_jobs=None, oob_score=False,
                    random_state=None, verbose=0, warm_start=False)

In [ ]:
ypred_etr = etr.predict(X_test)

In [ ]:
# df = X_test.copy()
# df['ypred_etr'] = ypred_etr
# df.to_csv('ypred_etr.csv')
# !cp ypred_etr.csv "drive/My Drive/GreyAtomCapstone/"
# df=None

In [ ]:
# pickle_out = open("etr_model.pickle","wb")
# pickle.dump(etr, pickle_out)
# pickle_out.close()
# !cp etr_model.pickle "drive/My Drive/GreyAtomCapstone/"

In [ ]:
# example_dict = {1:"6",2:"2",3:"f"}

# pickle_out = open("dict.pickle","wb")
# pickle.dump(example_dict, pickle_out)
# pickle_out.close()
# !cp dict.pickle "drive/My Drive/GreyAtomCapstone/"

---

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
y_test.isnull().sum()

374

In [ ]:
print('XGBoost:')
score = xg_reg.score(X_test, y_test)
rmse = np.sqrt(mean_squared_error(y_test, ypred_xgb))
print(f'R2:{score}\nRMSE:{rmse}')

XGBoost:
R2:0.6127057733422691
RMSE:4969040.92601269


In [ ]:
# print('RF:')
# print(f'R2:{rf.score(X_test, y_test)}\nRMSE:{np.sqrt(mean_squared_error(y_test, ypred_rf))}')

RF:
R2:0.5715723515913891
RMSE:5226257.541018996


In [ ]:
# print('ETR:')
# print(f'R2:{etr.score(X_test, y_test)}\nRMSE:{np.sqrt(mean_squared_error(y_test, ypred_etr))}')

ETR:
R2:0.5337629272229472
RMSE:5451995.18833136


In [ ]:
#SVM for poly kernel

In [34]:
svr_rbf = SVR(kernel='rbf', C=1e3)
svr_poly = SVR(kernel='poly', C=1e3)

In [ ]:
svr_poly_fit = svr_poly.fit(X_train, y_train)
y_poly = svr_poly_fit.predict(X_test)

In [ ]:
df = x_test.copy()
df['ypred_svm_poly'] = y_poly
df.to_csv('ypred_svm_poly.csv')
!cp ypred_svm_poly.csv "drive/My Drive/GreyAtomCapstone/"
df=None

In [ ]:
pickle_out = open("svm_poly_model.pickle","wb")
pickle.dump(svr_poly, pickle_out)
pickle_out.close()
!cp svm_poly_model.pickle "drive/My Drive/GreyAtomCapstone/"